# Librerías

In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import  pywhatkit as kit
from datetime import datetime
from time import sleep
import os
import time

In [40]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
service = Service('/usr/bin/chromedriver')

Comedor Universitario Siberia
SELECCIONAR
Comedor ESC AGROTECNICA
SELECCIONAR
Comedor Universitario Área Salud
SELECCIONAR
Comedor Universitario FCEIA
SELECCIONAR
Comedor Universitario Zavalla
SELECCIONAR
Comedor Universitario Casilda
SELECCIONAR
Comedor Universitario Área Centro

In [41]:
cod_comedores = {
    'siberia' : 0,
    'agrotecnia' : 1,
    'salud' : 2,
    'fceia' : 3,
    'zaballa' : 4,
    'casilda' : 5,
    'centro' : 6,
}

In [42]:
cod_menues_salud = {
    'des' : 0,
    'alm_llevar_14' : 1,
    'alm_llevar_13' : 2,
    'alm_llevar_12' : 3,
    'alm_llevar_11' : 4,
    'alm_no_llevar_14' : 5,
    'alm_no_llevar_13' : 6,
    'alm_no_llevar_12' : 7,
    'mer' : 8,
    'cen_llevar' : 9,
    'cen_no_llevar_20' : 10,
    'cen_no_llevar_19.30' : 11,
}

In [43]:
class Reserva:
    def __init__(self,u_dni,u_clave):
        self.driver = webdriver.Chrome(options=chrome_options)
        self.url_login = 'https://comedores.unr.edu.ar/'
        self.url_reservas = 'https://comedores.unr.edu.ar/comedor-reserva/reservar'
        self.u_dni = u_dni
        self.u_clave = u_clave
        self.u_logueado = False
        self.comedor = None
        self.menu = None
    
    def cerrar_navegador(self):
        print('se cierra el navegador')
        self.driver.quit()
    
    def loguearse(self):
        # Abrir la página web
        try:
            self.driver.get(self.url_login)
            # Capturar el elemento formulario
            e_form = WebDriverWait(driver=self.driver, timeout=2).until(
                EC.presence_of_element_located(locator=(By.ID, "form-login"))
            )
            print('formulario recibido')
            # Capturar los elementos del formulario
            e_input_dni = e_form.find_element(by=By.NAME,value='form-login[dni]')
            e_input_clave = e_form.find_element(by=By.NAME,value='form-login[clave]')
            # Completar el forumalario
            e_input_dni.send_keys('34178096')
            e_input_clave.send_keys('facundo0301')
            # Enviar el formulario
            e_boton = e_form.find_element(by=By.NAME,value='botones[botonEnviar]')
            e_boton.click()
            WebDriverWait(self.driver, 10).until(
                EC.invisibility_of_element_located(locator=(By.ID, "form-login"))
            )
            self.driver.implicitly_wait(200)
            print('usuario logueado, redireccionando...')        
            # Ingresar a la sección de reserva
            self.u_logueado = True
        except Exception as e:
            print(f"Ocurrió un error al visitar la página: {e}.")
            self.cerrar_navegador()
    def ingresar_comedor(self,comedor):
        if not self.u_logueado: 
            print('para elegir un comedor primero hay que loguearse')
            return
        # Abrir la página web
        try:
            self.driver.get(self.url_reservas)
            e_contenedor_comedores = WebDriverWait(driver=self.driver, timeout=2).until(
                EC.presence_of_element_located(locator=(By.CLASS_NAME,'reservar-comedores-contenedor'))
            )
            e_botones_comedores = e_contenedor_comedores.find_elements(by=By.TAG_NAME,value='button')
            n_boton_comedor = cod_comedores[comedor]
            e_boton_salud = e_botones_comedores[n_boton_comedor]
            # Ingresar al comedor de Salud
            e_boton_salud.click()
            print('Ingreso al comedor',comedor)
            self.comedor = comedor
        except Exception as e:
            print(f"Ocurrió un error al visitar la página: {e}.")
            self.cerrar_navegador()
    def reservar(self,menu):
        if self.comedor is None: 
            print('para reservar un menú primero hay que elegir un comedor')
            return
        try:
            op_reservas = self.driver.find_elements(by=By.CLASS_NAME,value='reservar-servicio')
            n_op_mer = cod_menues_salud[menu]
            op_mer = op_reservas[n_op_mer]
            op_mer.click()
            self.driver.implicitly_wait(20)
            print(f'Avancemos en reservar el menú {menu}')
            e_resumen = self.driver.find_element(by=By.CLASS_NAME,value='reserva-resumen-card')
            print(e_resumen.text)
            e_dias = self.driver.find_elements(by=By.CLASS_NAME,value='calendario-dia-turno')
            for i,e_dia in enumerate(e_dias):
                valor = e_dia.get_attribute(name='data-bind')
                if valor:
                    e_dia.click()
                    wait = WebDriverWait(self.driver, 10)
                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "swal2-container")))
                    e_cruz = self.driver.find_element(By.CLASS_NAME, "swal2-x-mark")
                    if e_cruz: print('hay un cruz',e_cruz)
                    boton_reservar = self.driver.find_element(By.CSS_SELECTOR, ".swal2-confirm")
                    print('vamos bien')
                    #boton_reservar.click()
                    #wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, "swal2-container")))
                    self.cerrar_navegador()
                    break       
        except Exception as e:
            print(f"Ocurrió un error al visitar la página: {e}.")
            self.cerrar_navegador()

In [44]:
reserva = Reserva(u_dni='34178096',u_clave='facundo0301')
reserva.loguearse()
reserva.ingresar_comedor(comedor='salud')
reserva.reservar(menu='mer')

formulario recibido
usuario logueado, redireccionando...
Ingreso al comedor salud
Avancemos en reservar el menú mer
Información
Comedor Universitario Área Salud
MERIENDA
De 16:00 a 19:00
hay un cruz <selenium.webdriver.remote.webelement.WebElement (session="2f9b3236e00904670868180b8af5e2bf", element="f.F6C8720ABE45160BCB3A57E5D94444BC.d.0F029FC60F204062074E195239F2EE63.e.85")>
vamos bien
se cierra el navegador
